In [4]:
import json
from dashboard import DEX_LIST, DEX_LIQUIDITY_METRIC_MAP
from constants import *

# open all files in test_results, create a dict for each and a combined dict pool_dict
'''DEX_pool_dict = {}
for dex in DEX_LIST:
    with open(f'test_results/pool_dict_{dex}.json', 'r') as f:
        DEX_pool_dict[dex] = json.load(f)

# create a combined dict by merging all dicts in pool_dict
combined_dict = {}
for dex in DEX_LIST:
    for key in DEX_pool_dict[dex]:
        combined_dict[key] = DEX_pool_dict[dex][key]'''
        
# open pool_dict.json from test_results
with open('data/pool_dict.json', 'r') as f:
    pool_dict = json.load(f)

In [21]:
import string
import pandas as pd
import numpy as np
import glob
import time
import random
import os

MIN_LIQUIDITY_THRESHOLD = 1_000_000
MAX_LIQUIDITY_THRESHOLD = 200_000_000_000

# Initialize empty dictionary for storing token liquidity
token_liquidity = {}

def refresh_matrix(pool_dict):
    # Calculate total liquidity for each token
    for pool in pool_dict.values():
        token0_id = pool['token0']['id']
        token1_id = pool['token1']['id']
        
        # Ignore tokens with punctuation in their symbol, band-aid fix for synthetic and irreputable tokens
        if any(char in string.punctuation for char in pool['token0']['symbol']):
            continue
        if any(char in string.punctuation for char in pool['token1']['symbol']):
            continue
        # Ignore Balancer_V1 and Balancer_V2 pools, another band-aid fix as their USD price is not accurate
        if pool['protocol'] == BALANCER_V1 or pool['protocol'] == BALANCER_V2 or pool['protocol'] == DODO:
            continue

        liquidity = float(pool[DEX_LIQUIDITY_METRIC_MAP[pool['protocol']]])
        reserve0 = float(pool['reserve0'])
        reserve1 = float(pool['reserve1'])
        total_reserve = reserve0 + reserve1

        # Calculate and add liquidity to respective tokens
        if token0_id in token_liquidity:
            try:
                token_liquidity[token0_id]['liquidity'] += liquidity * (reserve0 / total_reserve)
            except ZeroDivisionError:
                token_liquidity[token0_id]['liquidity'] += 0
        else:
            try:
                token_liquidity[token0_id] = {'symbol': pool['token0']['symbol'], 'liquidity': liquidity * (reserve0 / total_reserve)}
            except ZeroDivisionError:
                token_liquidity[token0_id] = {'symbol': pool['token0']['symbol'], 'liquidity': 0}
        if token1_id in token_liquidity:
            try:
                token_liquidity[token1_id]['liquidity'] += liquidity * (reserve1 / total_reserve)
            except ZeroDivisionError:
                token_liquidity[token1_id]['liquidity'] += 0
        else:
            try:
                token_liquidity[token1_id] = {'symbol': pool['token1']['symbol'], 'liquidity': liquidity * (reserve1 / total_reserve)}
            except ZeroDivisionError:
                token_liquidity[token1_id] = {'symbol': pool['token1']['symbol'], 'liquidity': 0}

    # sort token_liquidity by liquidity
    sorted_token_liquidity = dict(sorted(token_liquidity.items(), key=lambda item: item[1]['liquidity'], reverse=True))

    # drop any tokens with liquidity less than LIQUIDITY_THRESHOLD
    trimmed_sorted_token_liquidity = {}

    # Iterate over sorted_token_liquidity dictionary
    for k, v in sorted_token_liquidity.items():
        # If the liquidity is greater than the threshold, add it to the new dictionary
        if v['liquidity'] > MIN_LIQUIDITY_THRESHOLD and v['liquidity'] < MAX_LIQUIDITY_THRESHOLD:
            trimmed_sorted_token_liquidity[k] = v

    # Initialize a new dictionary for the trimmed pool_dict
    trimmed_pool_dict = {}

    # Iterate over the original pool_dict
    for pool_id, pool in pool_dict.items():
        # Get the IDs of the tokens in the current pool
        token0_id = pool['token0']['id']
        token1_id = pool['token1']['id']

        # If either of the tokens is in trimmed_sorted_token_liquidity, add the pool to trimmed_pool_dict
        if token0_id in trimmed_sorted_token_liquidity or token1_id in trimmed_sorted_token_liquidity:
            trimmed_pool_dict[pool_id] = pool

    trimmed_sorted_pools = sorted(trimmed_pool_dict.values(), key=lambda pool: float(pool[DEX_LIQUIDITY_METRIC_MAP[pool['protocol']]]), reverse=True)

    # Drop extreme pools, another band-aid fix
    # trimmed_sorted_pool_dict = {k: v for k, v in trimmed_sorted_pools.items() if float(v[DEX_LIQUIDITY_METRIC_MAP[v['protocol']]]) > MIN_LIQUIDITY_THRESHOLD and float(v[DEX_LIQUIDITY_METRIC_MAP[v['protocol']]]) < MAX_LIQUIDITY_THRESHOLD}

    trimmed_sorted_pools = list(filter(lambda pool: float(pool[DEX_LIQUIDITY_METRIC_MAP[pool['protocol']]]) > MIN_LIQUIDITY_THRESHOLD and float(pool[DEX_LIQUIDITY_METRIC_MAP[pool['protocol']]]) < MAX_LIQUIDITY_THRESHOLD, trimmed_sorted_pools))

    # Create a dictionary to store prices and identify tokens
    price_dict = {} # Dictionary for storing prices
    exchange_dict = {} # Dictionary for storing exchanges
    movement_dict_5m = {}  # Dictionary for storing price movement percentages for 5 minutes
    movement_dict_1h = {}  # Dictionary for storing price movement percentages for 1 hour
    movement_dict_24h = {}  # Dictionary for storing price movement percentages for 24 hours
    token_set = set() # Used to avoid duplicates
    TOKEN_SYMBOL_MAP = {} # Dictionary for storing token ids:symbols
    TOKEN_NAME_MAP = {} # Dictionary for storing token ids:names

    # Populate the dictionaries
    for pool in trimmed_sorted_pools:
        token0_id = pool['token0']['id']
        token1_id = pool['token1']['id']
        token0_symbol = pool['token0']['symbol']
        token1_symbol = pool['token1']['symbol']
        TOKEN_SYMBOL_MAP[token0_id] = token0_symbol
        TOKEN_SYMBOL_MAP[token1_id] = token1_symbol
        TOKEN_NAME_MAP[token0_id] = pool['token0']['name']
        TOKEN_NAME_MAP[token1_id] = pool['token1']['name']
        token_set.add(token0_id)
        token_set.add(token1_id)
        try:
            price0 = float(pool['token0'].get('priceUSD', 0))
            price1 = float(pool['token1'].get('priceUSD', 0))
            timestamp = time.time()  # Current time in seconds since the Epoch
            if price0 != 0 and price1 != 0:
                pair1 = (token0_id, token1_id)
                pair2 = (token1_id, token0_id)
                if pair1 not in price_dict:
                    price_dict[pair1] = []
                if pair2 not in price_dict:
                    price_dict[pair2] = []
                price_dict[pair1].append((timestamp, price1))
                price_dict[pair2].append((timestamp, price0))
                if pair1 not in exchange_dict:
                    exchange_dict[pair1] = []
                if pair2 not in exchange_dict:
                    exchange_dict[pair2] = []
                exchange_dict[pair1].append(pool['protocol'])
                exchange_dict[pair2].append(pool['protocol'])
        except Exception as e:
            print(e)

    movement_df_5m = pd.DataFrame(0, index=token_set, columns=token_set)  # DataFrame for storing price movement percentages
    movement_df_1h = pd.DataFrame(0, index=token_set, columns=token_set)  # DataFrame for storing 1-hour price movement percentages
    movement_df_24h = pd.DataFrame(0, index=token_set, columns=token_set)  # DataFrame for storing 24-hour price movement percentages

    # Compute the price movement percentage
    for pair, prices in price_dict.items():
        current_price = prices[-1][1]
        past_price_5m = None
        past_price_1h = None
        past_price_24h = None

        for timestamp, price in reversed(prices):
            if past_price_5m is None and timestamp <= time.time() - 60*5:  # Find the price 5 minutes ago
                past_price_5m = price
            if past_price_1h is None and timestamp <= time.time() - 60*60:  # Find the price 1 hour ago
                past_price_1h = price
            if past_price_24h is None and timestamp <= time.time() - 24*60*60:  # Find the price 24 hours ago
                past_price_24h = price
            if past_price_5m is not None and past_price_1h is not None and past_price_24h is not None:
                break

        movement_dict_5m[pair] = ((current_price - (past_price_5m or current_price)) / (past_price_5m or current_price)) * 100
        movement_dict_1h[pair] = ((current_price - (past_price_1h or current_price)) / (past_price_1h or current_price)) * 100
        movement_dict_24h[pair] = ((current_price - (past_price_24h or current_price)) / (past_price_24h or current_price)) * 100


    # Create an empty DataFrame for storing liquidity data
    liquidity_df = pd.DataFrame(0, index=token_set, columns=token_set) # DataFrame for storing liquidity
    # Create an empty DataFrame for storing volume data
    volume_df_24h = pd.DataFrame(0, index=token_set, columns=token_set)
    volume_df_1h = pd.DataFrame(0, index=token_set, columns=token_set)
    # Dict for storing which pools pairs can be found in
    pool_dict_for_pairs = {}

    # Populate the DataFrames
    for pool in trimmed_sorted_pools:
        token0_id = pool['token0']['id']
        token1_id = pool['token1']['id']

        if token0_id in TOKEN_SYMBOL_MAP and token1_id in TOKEN_SYMBOL_MAP:
            try:
                # Compute liquidity
                liquidity = float(pool[DEX_LIQUIDITY_METRIC_MAP[pool['protocol']]])
                liquidity_df.loc[token0_id, token1_id] += liquidity
                liquidity_df.loc[token1_id, token0_id] += liquidity

                # Compute volume
                volume_24h = float(pool['volume_24h'])
                volume_1h = float(pool['volume_1h'])
                volume_df_24h.loc[token0_id, token1_id] += volume_24h
                volume_df_24h.loc[token1_id, token0_id] += volume_24h
                volume_df_1h.loc[token0_id, token1_id] += volume_1h
                volume_df_1h.loc[token1_id, token0_id] += volume_1h
                
                # Compute price ratio
                price0 = float(pool['token0'].get('priceUSD', 0))
                price1 = float(pool['token1'].get('priceUSD', 0))

                if price0 != 0 and price1 != 0:
                    pair1 = (token0_id, token1_id)
                    pair2 = (token1_id, token0_id)
	
                    if pair1 not in pool_dict_for_pairs:
                        pool_dict_for_pairs[pair1] = []
                    if pair2 not in pool_dict_for_pairs:
                        pool_dict_for_pairs[pair2] = []
                    pool_dict_for_pairs[pair1].append(pool['id'])
                    pool_dict_for_pairs[pair2].append(pool['id'])

                    movement_df_5m.loc[pair1] = movement_dict_5m.get(pair1, 0)
                    movement_df_5m.loc[pair2] = movement_dict_5m.get(pair2, 0)
                    movement_df_1h.loc[pair1] = movement_dict_1h.get(pair1, 0)
                    movement_df_1h.loc[pair2] = movement_dict_1h.get(pair2, 0)
                    movement_df_24h.loc[pair1] = movement_dict_24h.get(pair1, 0)
                    movement_df_24h.loc[pair2] = movement_dict_24h.get(pair2, 0)

            except Exception as e:
                # print(f"Error with token pair ({TOKEN_SYMBOL_MAP[token0_id]}, {TOKEN_SYMBOL_MAP[token1_id]}): {e}")
                continue

    # Convert tuple keys to string
    str_price_dict = {str(k): v for k, v in price_dict.items()}

    # Save price_dict to data
    with open('data/price_dict.json', 'w') as f:
        json.dump(str_price_dict, f)

    # Create a DataFrame for average prices
    average_price_df = pd.DataFrame(np.nan, index=token_set, columns=token_set)
    movement_df_5m.replace(0, np.nan, inplace=True)  # Replace 0s with NaN for calculating percentage movements
    movement_df_1h.replace(0, np.nan, inplace=True)  # Replace 0s with NaN for calculating percentage movements
    movement_df_24h.replace(0, np.nan, inplace=True)  # Replace 0s with NaN for calculating percentage movements

    # Compute IQR for outlier detection for each pair
    trimmed_average_price_dict = {}
    limit = 1.5

    for k, v in price_dict.items():
        if v: # if there are prices
            Q1 = np.quantile(v, 0.25)
            Q3 = np.quantile(v, 0.75)
            IQR = Q3 - Q1

            # Define bounds for outliers
            lower_bound = Q1 - limit * IQR
            upper_bound = Q3 + limit * IQR

            # Trim outliers
            trimmed_prices = [price[1] for price in v if lower_bound <= price[1] <= upper_bound]
            # check if there are any trimmed prices using a.any()
            if trimmed_prices: # avoid division by zero
                trimmed_average_price_dict[k] = sum(trimmed_prices) / len(trimmed_prices)

    # Convert tuple keys to string
    str_trimmed_average_price_dict = {str(k): v for k, v in trimmed_average_price_dict.items()}

    # Populate the average_price_df using the trimmed_average_price_dict
    for (token0, token1), average_price in trimmed_average_price_dict.items():
        average_price_df.loc[token0, token1] = average_price

    # Save the trimmed average price dictionary to a file with a timestamp
    unix_timestamp_str = str(int(time.time()))
    trimmed_average_price_dict_path = f'data/trimmed_average_price_dict_{unix_timestamp_str}.json'
    with open(trimmed_average_price_dict_path, 'w') as f:
        json.dump(str_trimmed_average_price_dict, f)

    # Find the file that was created closest to 5 minutes ago
    filepaths = glob.glob('data/trimmed_average_price_dict_*.json')

    # Compute the price movement percentage using the current prices and the prices from 5 minutes, 1 hour and 24 hours ago
    closest_filepath_5m, closest_filepath_1h, closest_filepath_24h = None, None, None
    closest_diff_5m, closest_diff_1h, closest_diff_24h = None, None, None
    for filepath in filepaths:
        file_timestamp_str = filepath.split('_')[-1].split('.')[0]
        file_timestamp = int(file_timestamp_str)
        diff_5m = abs(time.time() - file_timestamp - 60*5)
        diff_1h = abs(time.time() - file_timestamp - 60*60)
        diff_24h = abs(time.time() - file_timestamp - 60*60*24)

        # Delete any files older than 25 hours
        if diff_24h > 60*60*25:
            os.remove(filepath)

        if closest_diff_5m is None or diff_5m < closest_diff_5m:
            closest_diff_5m = diff_5m
            closest_filepath_5m = filepath
        if closest_diff_1h is None or diff_1h < closest_diff_1h:
            closest_diff_1h = diff_1h
            closest_filepath_1h = filepath
        if closest_diff_24h is None or diff_24h < closest_diff_24h:
            closest_diff_24h = diff_24h
            closest_filepath_24h = filepath

    past_price_dict_5m = {}
    if closest_filepath_5m:
        with open(closest_filepath_5m, 'r') as f:
            past_price_dict_5m = json.load(f)
    past_price_dict_1h = {}
    if closest_filepath_1h:
        with open(closest_filepath_1h, 'r') as f:
            past_price_dict_1h = json.load(f)
    past_price_dict_24h = {}
    if closest_filepath_24h:
        with open(closest_filepath_24h, 'r') as f:
            past_price_dict_24h = json.load(f)

    for (token0, token1), current_price in trimmed_average_price_dict.items():
        past_price_5m = past_price_dict_5m.get(str((token0, token1)), current_price)
        price_movement_5m = ((current_price - past_price_5m) / past_price_5m) * 100
        movement_df_5m.loc[token0, token1] = price_movement_5m
        past_price_1h = past_price_dict_1h.get(str((token0, token1)), current_price)
        price_movement_1h = ((current_price - past_price_1h) / past_price_1h) * 100
        movement_df_1h.loc[token0, token1] = price_movement_1h
        past_price_24h = past_price_dict_24h.get(str((token0, token1)), current_price)
        price_movement_24h = ((current_price - past_price_24h) / past_price_24h) * 100
        movement_df_24h.loc[token0, token1] = price_movement_24h

    # Compute total liquidity for each token
    all_ids = list(set(liquidity_df.columns).union(set(liquidity_df.index)))
    row_sums = liquidity_df.sum(axis=0).reindex(all_ids, fill_value=0)
    col_sums = liquidity_df.sum(axis=1).reindex(all_ids, fill_value=0)
    liquidity_totals = row_sums + col_sums

    # Sort by total liquidity
    sorted_ids = liquidity_totals.sort_values(ascending=False).index

    # Reindex DataFrames according to sorted liquidity
    liquidity_df = liquidity_df.reindex(index=sorted_ids, columns=sorted_ids)
    average_price_df = average_price_df.reindex(index=sorted_ids, columns=sorted_ids)
    movement_df_5m = movement_df_5m.reindex(index=sorted_ids, columns=sorted_ids)
    movement_df_1h = movement_df_1h.reindex(index=sorted_ids, columns=sorted_ids)
    movement_df_24h = movement_df_24h.reindex(index=sorted_ids, columns=sorted_ids)

    # Remove the diagonal values as they don't represent valid token pairs
    np.fill_diagonal(liquidity_df.values, np.nan)
    np.fill_diagonal(average_price_df.values, np.nan)
    np.fill_diagonal(movement_df_5m.values, np.nan)
    np.fill_diagonal(movement_df_1h.values, np.nan)
    np.fill_diagonal(movement_df_24h.values, np.nan)

    # replace all nans with None
    liquidity_df = liquidity_df.where(pd.notnull(liquidity_df), None)
    average_price_df = average_price_df.where(pd.notnull(average_price_df), None)
    movement_df_5m = movement_df_5m.where(pd.notnull(movement_df_5m), None)
    movement_df_1h = movement_df_1h.where(pd.notnull(movement_df_1h), None)
    movement_df_24h = movement_df_24h.where(pd.notnull(movement_df_24h), None)

    # Create a combined DataFrame
    combined_df = pd.DataFrame(index=liquidity_df.index, columns=liquidity_df.columns)

    for row_id in combined_df.index:
        for col_id in combined_df.columns:
            liquidity = liquidity_df.loc[row_id, col_id] if pd.notnull(liquidity_df.loc[row_id, col_id]) else 0
            avg_price = average_price_df.loc[row_id, col_id] if pd.notnull(average_price_df.loc[row_id, col_id]) else 0
            price_movement_5m = movement_df_5m.loc[row_id, col_id] if pd.notnull(movement_df_5m.loc[row_id, col_id]) else 0
            price_movement_1h = movement_df_1h.loc[row_id, col_id] if pd.notnull(movement_df_1h.loc[row_id, col_id]) else 0
            price_movement_24h = movement_df_24h.loc[row_id, col_id] if pd.notnull(movement_df_24h.loc[row_id, col_id]) else 0
            volume_24h = volume_df_24h.loc[row_id, col_id]
            volume_1h = volume_df_1h.loc[row_id, col_id]
            safety_score = random.randrange(0, 5)
            pair = {}
            pair[row_id] = {
                'id': row_id,  
                'symbol': TOKEN_SYMBOL_MAP[row_id],
                'name': TOKEN_NAME_MAP[row_id],
            }
            pair[col_id] = {
                'id': col_id,  
                'symbol': TOKEN_SYMBOL_MAP[col_id],
                'name': TOKEN_NAME_MAP[col_id],
            }

            combined_df.at[row_id, col_id] = {
                'pair': pair,
                'liquidity': liquidity,
                'average_price': avg_price,
                'price_movement_5m': price_movement_5m,
                'price_movement_1h': price_movement_1h,
                'price_movement_24h': price_movement_24h,
                'volume_1h': volume_1h,
                'volume_24h': volume_24h,
                'safety_score': safety_score,
                'exchanges': set(exchange_dict.get((row_id, col_id), [])),
                'pools': list(set(pool_dict_for_pairs.get((row_id, col_id), [])))
            }
            # check if the cell is a diagonal
            if row_id == col_id:
                combined_df.at[row_id, col_id]['diagonal'] = True
                # set all values to 0
                combined_df.at[row_id, col_id]['liquidity'] = 0
                combined_df.at[row_id, col_id]['average_price'] = 0
                combined_df.at[row_id, col_id]['price_movement_5m'] = 0
                combined_df.at[row_id, col_id]['price_movement_1h'] = 0
                combined_df.at[row_id, col_id]['price_movement_24h'] = 0
                combined_df.at[row_id, col_id]['volume_24h'] = 0
                combined_df.at[row_id, col_id]['safety_score'] = 0
                combined_df.at[row_id, col_id]['exchanges'] = []
                combined_df.at[row_id, col_id]['pools'] = []
            else:
                combined_df.at[row_id, col_id]['diagonal'] = False
            # if the pool is not found in any pools or exchanges then any nonzero data is invalid
            if combined_df.at[row_id, col_id]['pools'] == [] or combined_df.at[row_id, col_id]['exchanges'] == []:
                combined_df.at[row_id, col_id]['liquidity'] = 0
                combined_df.at[row_id, col_id]['average_price'] = 0
                combined_df.at[row_id, col_id]['price_movement_5m'] = 0
                combined_df.at[row_id, col_id]['price_movement_1h'] = 0
                combined_df.at[row_id, col_id]['price_movement_24h'] = 0
                combined_df.at[row_id, col_id]['volume_24h'] = 0
                combined_df.at[row_id, col_id]['safety_score'] = 0
    # combined_df.to_csv('data/combined_df.csv')
    combined_df.to_json('data/combined_df_liquidity.json', orient='split')

    combined_df['mean_avg_price'] = combined_df.applymap(lambda x: x['average_price']).mean(axis=1)
    combined_df.sort_values(by=['mean_avg_price'], ascending=False, inplace=True)
    del combined_df['mean_avg_price']
    combined_df = combined_df.reindex(combined_df.index, axis=1)
    combined_df.to_json('data/combined_df_average_price.json', orient='split')

    combined_df['max_volume_24h'] = combined_df.applymap(lambda x: x['volume_24h']).max(axis=1)
    combined_df.sort_values(by=['max_volume_24h'], ascending=False, inplace=True)
    del combined_df['max_volume_24h']
    combined_df = combined_df.reindex(combined_df.index, axis=1)
    combined_df.to_json('data/combined_df_volume_24h.json', orient='split')

    # Move USDT, USDC, and DAI to the top 3 rows
    combined_df = combined_df.reindex(['0xdac17f958d2ee523a2206206994597c13d831ec7', '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', '0x6b175474e89094c44da98b954eedeac495271d0f'] + list(combined_df.index.difference(['0xdac17f958d2ee523a2206206994597c13d831ec7', '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', '0x6b175474e89094c44da98b954eedeac495271d0f'])), axis=0)

    # Make the columns have the same order as the rows
    combined_df = combined_df.reindex(combined_df.index, axis=1)

    combined_df.to_json('data/combined_df_popular.json', orient='split')
    
    return combined_df

In [22]:
combined_df = refresh_matrix(pool_dict)

In [23]:
combined_df.iloc[1,0]

{'pair': {'0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': {'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
   'symbol': 'USDC',
   'name': 'USD Coin'},
  '0xdac17f958d2ee523a2206206994597c13d831ec7': {'id': '0xdac17f958d2ee523a2206206994597c13d831ec7',
   'symbol': 'USDT',
   'name': 'Tether USD'}},
 'liquidity': 564318495.5915989,
 'average_price': 1.0000892231676672,
 'price_movement_5m': 0.0,
 'price_movement_1h': 0.0,
 'price_movement_24h': 0.0,
 'volume_1h': 232114.66257084618,
 'volume_24h': 53428333.701350346,
 'safety_score': 3,
 'exchanges': {'Balancer_V2', 'Curve'},
 'pools': ['0xbd3a698826d27563d08d459faff2d5f6960e21cf',
  '0x6aebf3f0829347b310d16b4c03a1d3b3838496aa',
  '0xf4e806b0fd2b278bd45b3bffe66c2c77bcf700be',
  '0xc3141fc45791cca3f21f2a926fd8598c39a4c6d2',
  '0x664e158dc6d1c0aacec01c59a9d11643f1a97e8a',
  '0x9f383f91c89cbd649c700c2bf69c2a828af299aa',
  '0x81b7f92c7b7d9349b989b4982588761bfa1aa627',
  '0x09a4551eea71d368358fa42acc9bed8d51566f2e',
  '0x0d70f2fc0096b9e3d

In [24]:
# this cell assumes you have a file called 'trimmed_average_price_dict.json' in the data folder 
# which can be generated by running the above cells then removing the timestamp

def randomize_prices(price_dict, lower, upper):
    for key, price in price_dict.items():
        random_change = random.uniform(lower, upper)  # Simulate up to +/-5% change
        price_dict[key] += price * random_change
        # price_dict[key] = max(price_dict[key], 0)  # Prevent negative prices
    return price_dict

# delete existing files begining with 'trimmed_average_price_dict_'
for filename in os.listdir('data'):
    if filename.startswith('trimmed_average_price_dict_'):
        os.remove(f'data/{filename}')

with open('data/trimmed_average_price_dict.json', 'r') as f:
    price_dict = json.load(f)
# timestamp 5 minutes ago
time_5m_ago = str(int(time.time()) - 300)
# timestamp 1 hour ago
time_1h_ago = str(int(time.time()) - 3600)
# timestamp 24 hours ago
time_24h_ago = str(int(time.time()) - 86400)
# randomize then save with timestamp 5 minutes ago
price_dict = randomize_prices(price_dict, -0.05, 0)
with open(f'data/trimmed_average_price_dict_{time_5m_ago}.json', 'w') as f:
    json.dump(price_dict, f)
# randomize then save with timestamp 1 hour ago
price_dict = randomize_prices(price_dict, 0.05, 0.1)
with open(f'data/trimmed_average_price_dict_{time_1h_ago}.json', 'w') as f:
    json.dump(price_dict, f)
# randomize then save with timestamp 24 hours ago
price_dict = randomize_prices(price_dict, 0.1, 0.5)
with open(f'data/trimmed_average_price_dict_{time_24h_ago}.json', 'w') as f:
    json.dump(price_dict, f)
    
df = refresh_matrix(pool_dict)
df.iloc[1,0]

{'pair': {'0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': {'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
   'symbol': 'USDC',
   'name': 'USD Coin'},
  '0xdac17f958d2ee523a2206206994597c13d831ec7': {'id': '0xdac17f958d2ee523a2206206994597c13d831ec7',
   'symbol': 'USDT',
   'name': 'Tether USD'}},
 'liquidity': 564318495.5915989,
 'average_price': 1.0000892231676672,
 'price_movement_5m': 3.241329714160821,
 'price_movement_1h': -4.9102702735792825,
 'price_movement_24h': -31.925818263578044,
 'volume_1h': 232114.66257084618,
 'volume_24h': 53428333.701350346,
 'safety_score': 2,
 'exchanges': {'Balancer_V2', 'Curve'},
 'pools': ['0xbd3a698826d27563d08d459faff2d5f6960e21cf',
  '0x6aebf3f0829347b310d16b4c03a1d3b3838496aa',
  '0xf4e806b0fd2b278bd45b3bffe66c2c77bcf700be',
  '0xc3141fc45791cca3f21f2a926fd8598c39a4c6d2',
  '0x664e158dc6d1c0aacec01c59a9d11643f1a97e8a',
  '0x9f383f91c89cbd649c700c2bf69c2a828af299aa',
  '0x81b7f92c7b7d9349b989b4982588761bfa1aa627',
  '0x09a4551eea71d368358

In [6]:
# find any pools containing the token 0xBB0E17EF65F82Ab018d8EDd776e8DD940327B28b, check token0 id and token1 id
pancake_pools = {}
for pool in pool_dict:
    if pool_dict[pool]['protocol'] == PANCAKESWAP_V3:
        pancake_pools[pool] = pool_dict[pool]

In [13]:
# find this pool in pancake_pools (pool['id']) 0x14bf727f67aa294ec36347bd95aba1a2c136fe7a
for pool in pancake_pools:
    if pancake_pools[pool]['token0']['symbol'] == 'rETH':
        print(pancake_pools[pool]['id'])

0x2201d2400d30bfd8172104b4ad046d019ca4e7bd


In [18]:
for pool in pancake_pools:
    if pancake_pools[pool]['id'] == '0x2201d2400d30bfd8172104b4ad046d019ca4e7bd':
        a = pancake_pools[pool]
        
a

{'token0': {'id': '0xae78736cd615f374d3085123a210448e74fc6393',
  'symbol': 'rETH',
  'name': 'Rocket Pool ETH',
  'decimals': '18',
  'priceUSD': 9650.03247550595},
 'token1': {'symbol': 'WETH',
  'name': 'Wrapped Ether',
  'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
  'decimals': '18',
  'priceUSD': 3756.3932191782924},
 'id': '0x2201d2400d30bfd8172104b4ad046d019ca4e7bd',
 'totalValueLockedUSD': '11333901.07073927639756095699101033',
 'liquidity': '7664164044945939780261294',
 'token0Price': '0.9289361843887342559924706423400394',
 'token1Price': '1.076500212614742425377922025008424',
 'feeTier': '500',
 'sqrtPrice': '82202806234021784419778492760',
 'volumeUSD': '15130240.91741341145297878051023574',
 'poolHourData': [{'volumeUSD': '1162320.477473803226683286061690536'}],
 'poolDayData': [{'volumeUSD': '1444678.028438683414977492298177048'}],
 'protocol': 'PancakeSwap_V3',
 'reserve0': '1174.493567716728451839',
 'reserve1': '3017.229669373793185258',
 'volume_24h': 0,
 'vol